In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://wm598-921.fios-router.home:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.33-5d8cae649505
LOGGING: writing to /Users/shifa/dev/hail_elasticsearch_pipelines/genome_sv_pipeline/hail-20210330-0920-0.2.33-5d8cae649505.log


In [2]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [3]:
# hl.import_vcf('vcf/sv.vcf.gz', force=True, reference_genome='GRCh38').write('vcf/svs.mt', overwrite=True)

In [4]:
mt = hl.read_matrix_table('vcf/svs.mt')

In [5]:
mt.count()

(145568, 714)

In [10]:
import subprocess
GS_SAMPLE_PATH = 'gs://seqr-datasets/v02/GRCh38/RDG_{sample_type}_Broad_Internal/base/projects/{project_guid}/{project_guid}_{file_ext}'
def _get_gs_samples(project_guid, file_ext, expected_header, sample_type):
    """
    Get sample metadata from files in google cloud

    :param project_guid: seqr project identifier
    :param file_ext: extension for the desired sample file
    :param expected_header: expected header to validate file
    :param sample_type: sample type (WES/WGS)
    :return: parsed data from the sample file as a list of lists
    """
    file = GS_SAMPLE_PATH.format(project_guid=project_guid, sample_type=sample_type, file_ext=file_ext)
    process = subprocess.Popen(
        'gsutil cat {}'.format(file), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    if process.wait() != 0:
        return None
    header = next(process.stdout).decode('utf-8')
    if header.strip() != expected_header:
        raise Exception('Missing header for sample file, expected "{}" but found {}'.format(
            expected_header, header))
    return [line.decode('utf-8').strip().split('\t') for line in process.stdout]


def get_sample_subset(project_guid, sample_type):
    """
    Get sample id subset for a given project

    :param project_guid: seqr project identifier
    :param sample_type: sample type (WES/WGS)
    :return: set of sample ids
    """
    subset = _get_gs_samples(project_guid, file_ext='ids.txt', sample_type=sample_type, expected_header='s')
    if not subset:
        raise Exception('No sample subset file found')
    return {row[0] for row in subset}

<br/>
Subset the samples to the project `R0332_cmg_estonia_wgs`

In [11]:
sample_subset = get_sample_subset('R0332_cmg_estonia_wgs', 'WGS')
subset = hl.literal(sample_subset)
mt1 = mt.filter_cols(subset.contains(mt['s']))
missing_samples = sample_subset - {col.s for col in mt1.key_cols_by().cols().collect()}
print('{} missing samples and the first 10 of them are: {}'.format(len(missing_samples), list(missing_samples)[:10]))

61 missing samples and the first 10 of them are: ['HK017-0046', 'OUN_HK132_002_D1', 'HK100-004_D1', 'OUN_HK124_001_D1', 'OUN_HK126_002_D1', 'HK100-002_D1', 'HK035_0089', 'HK060-0154_1', 'OUN_HK131_001_D1', 'OUN_HK126_001_D1']


In [12]:
samples = hl.agg.filter(mt1.GT.is_non_ref(), hl.agg.collect(hl.struct(id=mt1.s, gq=mt1.GQ, cn=mt1.RD_CN)))
mt2a = mt1.annotate_rows(samples=samples)
mt2 = mt2a.filter_rows(mt2a.samples != hl.empty_array(hl.dtype('struct{id: str, gq: int32, cn: int32}')))
mt2.count()

(67275, 106)

In [13]:
mtx = mt2.filter_entries(hl.is_defined(mt2.RD_CN))
rd_cn_cnt = sorted(mtx.aggregate_entries(hl.agg.counter(mtx.RD_CN)).items())
print('{} different RD_CNs, the counter for the first 10 RD_CNs are: {}.'.format(len(rd_cn_cnt), rd_cn_cnt[:10]))
print('Total RD_CNs>9: {}, max RD_CN: {}'.format(sum(rd_cn_cnt[10:][1]), rd_cn_cnt[-1][0]))

1181 different RD_CNs, the counter for the first 10 RD_CNs are: [(0, 51131), (1, 425325), (2, 4177497), (3, 315015), (4, 79143), (5, 32265), (6, 18043), (7, 11620), (8, 7940), (9, 6123)].
Total RD_CNs>9: 3797, max RD_CN: 4066


<br/>
The distributions of the values of the RD_CNs

In [14]:
dp_hist = mt2.aggregate_entries(hl.expr.aggregators.hist(mt2.RD_CN, start=0, end=10, bins=11))
p = hl.plot.histogram(dp_hist, legend='RD_CN', title='RD_CN Histogram')
show(p)

<br/>
Statistics for differet GT values

In [15]:
mt2.aggregate_entries(hl.agg.counter(mt2.GT))

{Call(alleles=[0, 0], phased=False): 6033770,
 Call(alleles=[0, 1], phased=False): 932341,
 Call(alleles=[1, 1], phased=False): 157472,
 None: 7567}

<br/>
RD_CN distributions for different GTs:

In [16]:
cnts={}
mtx = mt2.filter_entries(hl.is_defined(mt2.RD_CN))
for alts in range(3):
    field_name = 'GT'+str(alts)+'_RD_CN'
    mtx = mtx.annotate_entries(**{field_name: hl.if_else(mtx.GT.n_alt_alleles()==alts, mtx.RD_CN, -1)})
    cnts = sorted(mtx.aggregate_entries(hl.agg.counter(mtx[field_name])).items())
    cnts[11] = ('>9', sum(cnts[11:][1]))
    print('{}: {}'.format(field_name, cnts[:12]))

for field_name in ['GT0_RD_CN', 'GT1_RD_CN', 'GT2_RD_CN']:
    mtxx = mtx.filter_entries(mtx[field_name]>=0)
    dp_hist = mtxx.aggregate_entries(hl.expr.aggregators.hist(mtxx[field_name], start=0, end=10, bins=11))
    p = hl.plot.histogram(dp_hist, legend=field_name, title=field_name+' Histogram')
    show(p)

GT0_RD_CN: [(-1, 660908), (0, 8490), (1, 223817), (2, 3953419), (3, 199083), (4, 43557), (5, 20124), (6, 12082), (7, 7929), (8, 5431), (9, 4435), ('>9', 2885)]
GT1_RD_CN: [(-1, 4624911), (0, 14630), (1, 166020), (2, 206596), (3, 108528), (4, 22531), (5, 7503), (6, 3832), (7, 2535), (8, 1809), (9, 1244), ('>9', 668)]
GT2_RD_CN: [(-1, 5050345), (0, 28011), (1, 35488), (2, 17482), (3, 7404), (4, 13055), (5, 4638), (6, 2129), (7, 1156), (8, 700), (9, 444), ('>9', 266)]


In [17]:
mtx = mt2.filter_entries(hl.is_defined(mt2.RD_CN))
for cn in range(10):
    field_name = 'RD_CN_{}'.format(cn)
    mtx = mtx.annotate_entries(**{field_name: hl.if_else(mtx.RD_CN==cn, hl.struct(alts=mtx.GT.n_alt_alleles(), svtype=mtx.info.SVTYPE),
                                                         hl.struct(alts=-1, svtype='NA'))})
    cnts = mtx.aggregate_entries(hl.agg.counter(mtx[field_name]))
    cnts = sorted(cnts.items(), key=lambda item: item[0].alts)
    pprint((field_name, cnts[1:]))

mtx = mtx.annotate_entries(**{'RD_CN_gt_9': hl.if_else(mtx.RD_CN>9, hl.struct(alts=mtx.GT.n_alt_alleles(), svtype=mtx.info.SVTYPE),
                                                         hl.struct(alts=-1, svtype='NA'))})
cnts = mtx.aggregate_entries(hl.agg.counter(mtx.RD_CN_gt_9))
cnts = sorted(cnts.items(), key=lambda item: item[0].alts)
pprint(('RD_CN_gt_9', cnts[1:]))

('RD_CN_0',
 [(Struct(alts=0, svtype='DUP'), 3171),
  (Struct(alts=0, svtype='CPX'), 4),
  (Struct(alts=0, svtype='INS'), 16),
  (Struct(alts=0, svtype='DEL'), 5299),
  (Struct(alts=1, svtype='DUP'), 116),
  (Struct(alts=1, svtype='DEL'), 14487),
  (Struct(alts=1, svtype='INS'), 27),
  (Struct(alts=2, svtype='DUP'), 8),
  (Struct(alts=2, svtype='DEL'), 28002),
  (Struct(alts=2, svtype='CPX'), 1)])
('RD_CN_1',
 [(Struct(alts=0, svtype='DUP'), 86321),
  (Struct(alts=0, svtype='CPX'), 191),
  (Struct(alts=0, svtype='INS'), 402),
  (Struct(alts=0, svtype='DEL'), 136903),
  (Struct(alts=1, svtype='DUP'), 2011),
  (Struct(alts=1, svtype='DEL'), 163754),
  (Struct(alts=1, svtype='CPX'), 43),
  (Struct(alts=1, svtype='INS'), 212),
  (Struct(alts=2, svtype='DUP'), 11),
  (Struct(alts=2, svtype='DEL'), 35471),
  (Struct(alts=2, svtype='INS'), 6)])
('RD_CN_2',
 [(Struct(alts=0, svtype='DUP'), 2231604),
  (Struct(alts=0, svtype='CPX'), 2290),
  (Struct(alts=0, svtype='INS'), 6123),
  (Struct(alts=